# Calibration

This section of the tutorial will cover the calibration module of Mitiq. 

Before running this tutorial, make sure you have `mitiq` installed.
```
pip install mitiq
```

In [20]:
# If you have not already done so, you can install mitiq by uncommenting out the line below
# %pip install mitiq

Since we will be using Qiskit's FakeJakartaV2 simulator for this demonstration, it is necessary to install some of the Qiskit packages. This can be done by running the pip install command below.

In [21]:
%pip install qiskit qiskit-aer qiskit-ibm-runtime ply

Note: you may need to restart the kernel to use updated packages.


In [22]:
from mitiq.zne.inference import LinearFactory, RichardsonFactory

from mitiq import (
    Calibrator,
    MeasurementResult,
    Settings
)

from mitiq.zne.scaling import (
    fold_all,
    fold_gates_at_random,
    fold_global
)

import numpy as np
from qiskit_ibm_runtime.fake_provider import FakeJakartaV2  # Fake (simulated) QPU

## Executor for Calibration
When using the calibration module to test different mitigation techniques, you must define an executor that returns measured bitstrings. 

In [23]:
def execute_qiskit_FakeJakartaV2(circuit, shots=1000):
    """Return bitstrings outcomes from circuit executions performed by
    the FakeJakartaV2 simulator with noise."""
    noisy_backend = FakeJakartaV2()
    noisy_result = noisy_backend.run(circuit, shots=shots).result()
    noisy_counts = noisy_result.get_counts(circuit)
    noisy_counts = { k.replace(" ",""):v for k, v in noisy_counts.items()}
    measurements = MeasurementResult.from_counts(noisy_counts)
    return measurements

## Run the Calibrator
Now that the executor is defined, we can instantiate the Calibrator by passing in the executor along with a `fronted`. As a reminder of the available front ends

In [24]:
from mitiq import SUPPORTED_PROGRAM_TYPES
SUPPORTED_PROGRAM_TYPES.keys()

['braket', 'cirq', 'pennylane', 'pyquil', 'qibo', 'qiskit']

In [25]:
cal = Calibrator(execute_qiskit_FakeJakartaV2, frontend="qiskit")
cal.run(log="flat")

/Users/briangoldsmith/anaconda3/envs/mitiq/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiq/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiq/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiq/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiq/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/

┌──────────────────────────┬────────────────────────────────────┬────────────────────────────┐
│ benchmark                │ strategy                           │ performance                │
├──────────────────────────┼────────────────────────────────────┼────────────────────────────┤
│ Type: ghz                │ Technique: ZNE                     │ ✔                          │
│ Num qubits: 2            │ Factory: Linear                    │ Noisy error: 0.02          │
│ Circuit depth: 2         │ Scale factors: 1.0, 3.0, 5.0       │ Mitigated error: 0.0075    │
│ Two qubit gate count: 1  │ Scale method: fold_gates_at_random │ Improvement factor: 2.6667 │
├──────────────────────────┼────────────────────────────────────┼────────────────────────────┤
│ Type: ghz                │ Technique: ZNE                     │ ✔                          │
│ Num qubits: 2            │ Factory: Linear                    │ Noisy error: 0.02          │
│ Circuit depth: 2         │ Scale factors: 1.0, 2

In [26]:
cal.run(log="cartesian")

/Users/briangoldsmith/anaconda3/envs/mitiq/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiq/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiq/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiq/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiq/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/

┌────────────────────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────┐
│ strategy\benchmark                 │ Type: ghz                  │ Type: w                    │ Type: rb                   │ Type: mirror               │
│                                    │ Num qubits: 2              │ Num qubits: 2              │ Num qubits: 2              │ Num qubits: 2              │
│                                    │ Circuit depth: 2           │ Circuit depth: 2           │ Circuit depth: 54          │ Circuit depth: 33          │
│                                    │ Two qubit gate count: 1    │ Two qubit gate count: 2    │ Two qubit gate count: 12   │ Two qubit gate count: 14   │
├────────────────────────────────────┼────────────────────────────┼────────────────────────────┼────────────────────────────┼────────────────────────────┤
│ Technique: ZNE                     │ ✘                          │ ✘ 

In [27]:
cal.best_strategy()

{'technique': 'ZNE', 'factory': 'RichardsonFactory', 'scale_factors': [1.0, 3.0, 5.0], 'scale_method': 'fold_global'}

# SETTINGS
The settings parameter in the Calibrator allows for specifying details about circuit and technique. In this example, we will continue working with ZNE, but only look at the random benchmarking and mirror circuits.

The available options for the type of circuits is `ghz`, `w`, `rb`, and `mirror`. For ZNE, the `scale_noise` the options include `fold_global`, `fold_gates_at_random`, or `fold_all`. The ZNE factories for extrapolation include `RichardsonFactory` and `LinearFactory`.

In [28]:
CustomSettings = Settings(
    benchmarks=[
        {
            "circuit_type": "rb",
            "num_qubits": 2,
            "circuit_depth": 20,
            "two_qubit_gate_count": 10,
        },
        {
            "circuit_type": "rb",
            "num_qubits": 2,
            "circuit_depth": 50, 
            "two_qubit_gate_count": 20, 
        },
        {
            "circuit_type": "mirror",
            "num_qubits": 2,
            "circuit_depth": 20,
            "two_qubit_gate_count": 10,
        },
        {
            "circuit_type": "mirror",
            "num_qubits": 2,
            "circuit_depth": 50, 
            "two_qubit_gate_count": 20,
        },
    ],
    strategies=[
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": RichardsonFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": RichardsonFactory([1.0, 4.0, 7.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": LinearFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": LinearFactory([1.0, 4.0, 7.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": RichardsonFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": RichardsonFactory([1.0, 4.0, 7.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": LinearFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": LinearFactory([1.0, 4.0, 7.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_all,
            "factory": RichardsonFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_all,
            "factory": RichardsonFactory([1.0, 4.0, 7.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_all,
            "factory": LinearFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_all,
            "factory": LinearFactory([1.0, 4.0, 7.0]),
        },

    ],
)

In [29]:
cal = Calibrator(execute_qiskit_FakeJakartaV2, frontend="qiskit", settings=CustomSettings)
cal.run(log="flat")

┌───────────────────────────┬────────────────────────────────────┬────────────────────────────┐
│ benchmark                 │ strategy                           │ performance                │
├───────────────────────────┼────────────────────────────────────┼────────────────────────────┤
│ Type: rb                  │ Technique: ZNE                     │ ✔                          │
│ Num qubits: 2             │ Factory: Linear                    │ Noisy error: 0.11          │
│ Circuit depth: 137        │ Scale factors: 1.0, 2.0, 3.0       │ Mitigated error: 0.0133    │
│ Two qubit gate count: 35  │ Scale method: fold_all             │ Improvement factor: 8.25   │
├───────────────────────────┼────────────────────────────────────┼────────────────────────────┤
│ Type: rb                  │ Technique: ZNE                     │ ✔                          │
│ Num qubits: 2             │ Factory: Linear                    │ Noisy error: 0.11          │
│ Circuit depth: 137        │ Scale fact